In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
import random as rd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from nltk.stem.porter import *  

import re
import urllib.request
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = None
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

nlp = spacy.load("en_core_web_md")

from spacy import displacy
import df_helper as dfh
import nlp_support as ns


from sklearn.utils.random import sample_without_replacement
import importlib
import pickle
from sklearn.feature_selection import SelectKBest, chi2,  RFE, RFECV, SelectFpr

In [ ]:
importlib.reload(dfh)
importlib.reload(ns)

In [ ]:
from sklearn.base import TransformerMixin


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, validation_curve
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix

import sklearn.metrics as met
from sklearn.metrics import precision_recall_curve, roc_curve, auc, matthews_corrcoef
from sklearn.metrics import confusion_matrix, fbeta_score, make_scorer, average_precision_score, auc, \
    accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, classification_report, \
    brier_score_loss, roc_auc_score

from scipy.stats import randint as sp_randint , uniform
import eli5

from joblib import dump, load


In [ ]:
%%timeit -N 1

if 1:   
    
    X_train,Y_train,X_test, Y_test,X_valid, Y_valid= ns.load_data()
    n_false_sample= 2500
    n_features=100

    DOE='under_false_%s_nwords_%s_mindf_1' % (n_false_sample,n_features)
    print(DOE)

    data_set, bow_bal, tfidf = ns.train_bow(X_train,Y_train, X_test,Y_test,n_false_sample ,  max_df=0.9,min_df=1, max_features=n_features )

    with open('%s_data.pkl' % DOE, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_set, f)
        
        from joblib import dump, load
        dump(tfidf, '%s_tfidf.joblib' % DOE)
        dump(bow_bal, '%s_bal_cv.joblib' % DOE)
        
    print('Saved')

under_false_2500_nwords_100_mindf_1
Original n_true, n_false: 2329 18717
Resampled n_true, n_false: 2329 2500
Saved


C:\anaconda3\envs\ds\lib\timeit.py:206: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  t = self.timeit(number)


under_false_2500_nwords_100_mindf_1
Original n_true, n_false: 2329 18717
Resampled n_true, n_false: 2329 2500


In [4]:
with open('%s_data.pkl' % DOE, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data_set, f)

    from joblib import dump, load
    dump(tfidf, '%s_tfidf.joblib' % DOE)
    dump(bow_bal, '%s_bal_cv.joblib' % DOE)

print('Saved')

NameError: name 'DOE' is not defined

In [ ]:
%%timeit
if 1:

    X_train,Y_train,X_test, Y_test,X_valid, Y_valid= ns.load_data()
    n_false_sample= None
    n_features=5000

    DOE='fs_false_all_nwords_%s_mindf_1' % (n_false_sample,n_features)
    print(DOE)

    data_set2, bow_bal2, tfidf2 = ns.train_bow(X_train,Y_train, X_test,Y_test,n_false_sample ,  max_df=0.9,min_df=1, max_features=n_features )

    with open('%s_data.pkl' % DOE, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_set2, f)
        
        from joblib import dump, load
        dump(tfidf2, '%s_tfidf.joblib' % DOE)
        dump(bow_bal2, '%s_bal_cv.joblib' % DOE)
        
    print('Saved')